In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime, UTC

from typing import List

In [2]:
previousDate = datetime.strptime("16-09-2007", "%d-%m-%Y").astimezone(UTC).timestamp()
currentDate = datetime.strptime(datetime.strftime(datetime.today(), "%d-%m-%Y"), "%d-%m-%Y").astimezone(UTC).timestamp()

url = f"https://finance.yahoo.com/quote/%5ENSEI/history/?period1={previousDate}&period2={currentDate}"
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'
}

In [3]:
req = requests.get(url, headers=header)

In [4]:
soup = BeautifulSoup(req.content, 'html.parser')

In [5]:
table = soup.find_all('table')

In [6]:
tr = table[0].findAll("tr")

In [7]:
df = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])

k = 0
for i in range(len(tr)):
	td = tr[i].findAll("td")

	obj: List[str] = []

	for j in range(len(td)):
		if j == 0:
			obj.append(datetime.strptime(td[j].getText(), "%b %d, %Y").strftime("%d-%b-%Y"))
		else:
			try:
				obj.append(str(float(td[j].getText().replace(",",""))))
			except:
				obj.append("0")
	if len(obj) == len(df.columns):
		df.loc[k] = obj
		k += 1

In [8]:
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
df.sort_values("Date", ascending=False)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2025-03-07,22508.65,22633.8,22464.75,22552.5,22552.5,289800.0
1,2025-03-06,22476.35,22556.45,22245.85,22544.7,22544.7,372100.0
2,2025-03-05,22073.05,22394.9,22067.8,22337.3,22337.3,305900.0
3,2025-03-04,21974.45,22105.05,21964.6,22082.65,22082.65,253300.0
4,2025-03-03,22194.55,22261.55,22004.7,22119.3,22119.3,282400.0
...,...,...,...,...,...,...,...
4280,2007-09-21,4752.95,4855.7,4733.7,4837.55,4837.55,0
4281,2007-09-20,4734.85,4760.85,4721.15,4747.55,4747.55,0
4282,2007-09-19,4550.25,4739.0,4550.25,4732.35,4732.35,0
4283,2007-09-18,4494.1,4551.8,4481.55,4546.2,4546.2,0


In [10]:
oldNSEI = df[df['Date'] < datetime.strptime("01-01-2025", "%d-%m-%Y")]
newNSEI = df[df['Date'] >= datetime.strptime("01-01-2025", "%d-%m-%Y")]

df.to_csv("NSEI.csv", index=False)
oldNSEI.to_csv("Old NSEI.csv", index=False)
newNSEI.to_csv("New NSEI.csv", index=False)